Chat GPT code!

In [1]:
import cv2
import json
import os

# Specify the folder containing videos and output directories
video_folder = "C:\\awilde\\britta\\LTU\\SkiingProject\\SkiersProject\\Data\\test\\"
output_frame_path = "C:\\awilde\\britta\\LTU\\SkiingProject\\SkiersProject\\Data\\Frames_of_videos\\"
annotation_folder = "C:\\awilde\\britta\LTU\\SkiingProject\\SkiersProject\\Data\\Annotations\\"  # Folder with corresponding JSON files
combined_json_path = "C:\\awilde\\britta\LTU\\SkiingProject\\SkiersProject\\Data\\Annotations\\combined_annotations.json"

# Maximum frame limits for specific videos (optional)
frame_limits = {
    "DJI_0009.MP4": 890
}

# Initialize counters
frame_counter = 0
combined_data = {
    "licenses" : [],
    "info" : [],
    "images": [],
    "annotations": [],
    "categories": []
}

In [2]:
image_id_counter = 1  # Start image IDs at 1
annotation_id_counter = 1  # Start annotation IDs at 1

video_files = [f for f in os.listdir(video_folder) if f.endswith(('.mp4', '.MP4'))]

# Process each video and corresponding JSON
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    json_path = os.path.join(annotation_folder, f"{os.path.splitext(video_file)[0]}_annotations.json")

    # Check if the JSON file exists
    if not os.path.exists(json_path):
        print(f"Warning: JSON file not found for {video_file}. Skipping...")
        continue

    # Split video into frames
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    current_frame = 0
    max_frames = frame_limits.get(video_file, float('inf'))  # Default to no limit if not specified

    while success and current_frame <= max_frames:
        # Save frame with sequential numbering based on global frame_counter
        frame_name = f"frame_{frame_counter:06d}.jpg"
        save_path = os.path.join(output_frame_path, frame_name)
        cv2.imwrite(save_path, image)

        # Add to combined JSON
        combined_data["images"].append({
            "id": image_id_counter,
            "width": image.shape[1],
            "height": image.shape[0],
            "file_name": frame_name,
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        })

        success, image = vidcap.read()
        frame_counter += 1
        image_id_counter += 1  # Increment image ID counter
        current_frame += 1

    # Update JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)

    for annotation in data.get("annotations", []):
        # Assign a sequential ID and link to the correct image
        annotation["id"] = annotation_id_counter
        annotation["image_id"] = annotation_id_counter  # Align with the correct frame ID
        combined_data["annotations"].append(annotation)
        annotation_id_counter += 1

    # Add categories only once (assumes they are consistent across files)
    if not combined_data["categories"]:
        combined_data["categories"] = data.get("categories", [])

# Save the combined JSON
with open(combined_json_path, 'w') as f:
    json.dump(combined_data, f, indent=4)

print(f"Processed {frame_counter} frames in total.")

Processed 2446 frames in total.


# Verification

In [3]:
import random
import cv2

# Draw annotations on a frame
def draw_annotations(frame, annotations, categories):
    for ann in annotations:
        keypoints = ann.get("keypoints", [])
        category_id = ann.get("category_id", None)
        category_name = next((cat["name"] for cat in categories if cat["id"] == category_id), "unknown")
        
        # Draw keypoints
        for i in range(0, len(keypoints), 3):
            x, y, v = keypoints[i:i+3]  # x, y, visibility
            if v > 0:  # Only draw visible keypoints
                cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)  # Green keypoints

        # Optionally, draw lines (skeleton) if available
        skeleton = categories[0].get("skeleton", [])
        for connection in skeleton:
            start_idx, end_idx = connection
            if start_idx < len(keypoints) // 3 and end_idx < len(keypoints) // 3:
                x1, y1, v1 = keypoints[start_idx * 3:start_idx * 3 + 3]
                x2, y2, v2 = keypoints[end_idx * 3:end_idx * 3 + 3]
                if v1 > 0 and v2 > 0:  # Only draw lines between visible points
                    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Blue lines

    return frame

# Verify frames by drawing annotations
def verify_frames_with_annotations(images, annotations, frame_path, categories, num_verifications=5):
    print("\n--- Verifying Frames with Annotations ---")
    for _ in range(num_verifications):
        # Pick a random image
        image = random.choice(images)
        frame_file = os.path.join(frame_path, image["file_name"])

        # Find associated annotations
        associated_annotations = [ann for ann in annotations if ann["image_id"] == image["id"]]

        print(f"Verifying Frame: {image['file_name']}, ID: {image['id']}")
        print("Annotations:")
        for ann in associated_annotations:
            print(f"  - ID: {ann['id']}, Keypoints: {ann['keypoints']}")

        # Load and annotate the frame
        frame = cv2.imread(frame_file)
        if frame is not None:
            annotated_frame = draw_annotations(frame, associated_annotations, categories)

            # Display the annotated frame
            cv2.imshow(f"Frame {image['id']} with Annotations", annotated_frame)
            cv2.waitKey(0)  # Press any key to close the frame window
            cv2.destroyAllWindows()
        else:
            print(f"Warning: Unable to load frame {frame_file}")

# Call verification after processing all videos
verify_frames_with_annotations(
    combined_data["images"], 
    combined_data["annotations"], 
    output_frame_path, 
    combined_data["categories"]
)


--- Verifying Frames with Annotations ---
Verifying Frame: frame_000291.jpg, ID: 292
Annotations:
  - ID: 292, Keypoints: [976.72, 731.44, 1, 978.77, 727.35, 1, 972.63, 727.35, 1, 984.9, 723.26, 1, 968.54, 723.26, 1, 995.13, 737.58, 1, 958.32, 735.53, 1, 1005.35, 758.03, 1, 944.0, 753.94, 1, 1003.31, 782.56, 1, 939.91, 780.52, 1, 982.86, 784.61, 1, 962.41, 782.56, 1, 980.21, 830.88, 1, 942.25, 818.48, 1, 980.56, 864.66, 1, 917.06, 846.02, 1, 976.72, 710.99, 1, 976.72, 731.44, 1, 972.63, 780.52, 1, 980.17, 878.83, 1, 907.19, 858.38, 1, 984.9, 876.63, 1, 904.95, 854.29, 1, 977.57, 867.01, 1, 917.93, 849.05, 1]
Verifying Frame: frame_000830.jpg, ID: 831
Annotations:
  - ID: 831, Keypoints: [577.42, 680.24, 1, 581.52, 674.09, 1, 573.32, 676.14, 1, 587.67, 674.09, 1, 569.22, 676.14, 1, 602.03, 690.5, 1, 565.11, 694.6, 1, 614.33, 719.21, 1, 563.06, 721.26, 1, 595.88, 727.41, 1, 567.16, 747.92, 1, 602.03, 743.82, 1, 583.57, 745.87, 1, 610.23, 788.94, 1, 585.62, 788.94, 1, 623.26, 832.47, 1, 

In [8]:
# import os
# import json
# import random
# import shutil

# # File paths (reloading after reset)
# json_file_path = 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated.json'
# images_folder = "E:\\SkiProject\\Frames_of_videos\\DJI_0044\\"  # Assuming images are in this folder
# train_folder = "E:\\SkiProject\\Frames_of_videos\\DJI_0044\\train\\"
# val_folder = "E:\\SkiProject\\Frames_of_videos\\DJI_0044\\val\\"

# # Load the JSON data
# with open(json_file_path, 'r') as f:
#     data = json.load(f)

# # Create train and validation folders
# os.makedirs(train_folder, exist_ok=True)
# os.makedirs(val_folder, exist_ok=True)

# # Shuffle and split the images (80% train, 20% validation)
# images = data["images"]
# random.shuffle(images)
# split_index = int(0.8 * len(images))
# train_images = images[:split_index]
# val_images = images[split_index:]

# # Helper function to move images
# def move_images(image_list, dest_folder):
#     for image in image_list:
#         image_name = image['file_name']
#         src_path = os.path.join(images_folder, image_name)
#         dest_path = os.path.join(dest_folder, image_name)
#         if os.path.exists(src_path):
#             shutil.copy(src_path, dest_path)

# # Move train and validation images to their respective folders
# move_images(train_images, train_folder)
# move_images(val_images, val_folder)

# # Filter annotations for train and validation
# train_image_ids = {img["id"] for img in train_images}
# val_image_ids = {img["id"] for img in val_images}

# train_annotations = [ann for ann in data["annotations"] if ann["image_id"] in train_image_ids]
# val_annotations = [ann for ann in data["annotations"] if ann["image_id"] in val_image_ids]

# # Create new train and validation JSON files
# train_data = {
#     "info": data["info"],
#     "licenses": data["licenses"],
#     "categories": data["categories"],
#     "images": train_images,
#     "annotations": train_annotations
# }

# val_data = {
#     "info": data["info"],
#     "licenses": data["licenses"],
#     "categories": data["categories"],
#     "images": val_images,
#     "annotations": val_annotations
# }

# # Save the train and validation JSON files
# train_json_path = 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated_train.json'
# val_json_path = 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated_val.json'

# with open(train_json_path, 'w') as f:
#     json.dump(train_data, f, indent=4)

# with open(val_json_path, 'w') as f:
#     json.dump(val_data, f, indent=4)

# train_json_path, val_json_path


('E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated_train.json',
 'E:\\SkiProject\\annotations_test_DJI_0044\\person_keypoints_default_updated_val.json')